In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('events_up_to_01062018.csv',low_memory=False)

In [3]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [4]:
def isnew (x):
    c = x['new_vs_returning'].value_counts().index
    return ('New' in c)
new = file.groupby('person').apply(isnew)

In [5]:
new.value_counts()

True     38242
False      587
dtype: int64

In [6]:
def region (x):
    c = x['region'].value_counts().index
    if len(c)>0:
        return c[0]
    else:
        return 'Unknown'
reg = file.groupby('person').apply(region)

In [7]:
reg.head()

person
0008ed71              Unknown
00091926    Rio Grande do Sul
00091a7a         Minas Gerais
000ba417            Sao Paulo
000c79fe         Minas Gerais
dtype: object

In [8]:
def viocelulares(x):
    return (x.loc[x.event=='viewed product']['event'].size > 0)
viocelulares = file.groupby('person').apply(viocelulares)

In [9]:
viocelulares.value_counts()

True     37130
False     1699
dtype: int64

In [10]:
features = pd.DataFrame(viocelulares).join(pd.DataFrame(reg),lsuffix='v').join(pd.DataFrame(new))
features.columns = ['vio_celulares','region','is_new']

In [11]:
features.head()

,vio_celulares,region,is_new
person,,,
0008ed71,False,Unknown,True
00091926,True,Rio Grande do Sul,True
00091a7a,True,Minas Gerais,True
000ba417,True,Sao Paulo,True
000c79fe,True,Minas Gerais,True


In [12]:
features_01 = pd.read_csv('features_01.csv')

In [13]:
features_03 = features_01.set_index('person').join(features,how='left')

In [14]:
features_03.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,visitas_por_dia,vio_celulares,region,is_new
person,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral,2.000000,False,Unknown,True
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct,1.214286,True,Rio Grande do Sul,True
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid,1.000000,True,Minas Gerais,True
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct,0.600000,True,Sao Paulo,True
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid,1.000000,True,Minas Gerais,True


In [15]:
features_03.to_csv('features_03.csv')